# Conversão Neo4J 
### Imports


In [ ]:
import cx_Oracle
import math
import numpy as np
import json

# path da instalação do instantclient
cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient_21_9")

### Estabelecer Conexão 


In [ ]:
# Conectar ao Oracle
dsn_tns = cx_Oracle.makedsn('localhost', '1521', service_name='xe')
conn = cx_Oracle.connect(user='store', password='uminho2023', dsn=dsn_tns)

cursor = conn.cursor()

### Criar nodos das Tabelas

Nota: É aconselhável
Variável count pois se quisermos meter várias queries no Neo4j ao mesmo tempo estas nao podem ter a mesma variável para a label


- `Store_Users`

In [ ]:
# Criar nodos dos Utilizadores
def createStoreUserNodes(cursor, cyph):
    query = "SELECT * FROM STORE_USERS"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela Store_Users
    count = 1
    for l in result:
        id_line = l[0]
        name = l[1]
        m_name = l[2]
        l_name = l[3]
        phone = l[4]
        email = l[5]
        username = l[6]
        password = l[7]
        registered = l[8]

        cyph.write('CREATE (u'+ str (count) + ':Store_User {id:' + str(id_line) + ', First_name:' + "'" + name + "'"+ ', Middle_name:' + "'" + str(m_name) + "'" + ', Last_name :' + "'" + str(l_name) + "'" + ', Phone_Number:' + "'" + str(phone) + "'" + ', Email:' + "'" + str(email) + "'" + ', Username:' + "'" + str(username) + "'" + ', User_password:' + "'" + str(password) + "'" + ', Registered_at:' + "'" + str(registered) + "'" +  '});'+'\n')
        count += 1
    cyph.write('\n\n')

- `Product`

In [ ]:
# Criar nodos dos Produtos
def createProductNodes(cursor, cyph):
    query = "SELECT * FROM PRODUCT"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela PRODUCT
    count = 1
    for l in result:
        id_line = l[0]
        name = l[1]
        category_id = l[2]
        sku = l[3]
        price = l[4]
        discount_id = 0
        if(l[5] is not None):
            discount_id = l[5]
        
        created_at = l[6]
        modified_str = '2022-07-01 09:57:48'
        if l[7] is not None:
            modified_str = str(l[7])

        # Obtenha o stock do produto a partir da tabela "STOCK"
        stock_query = "SELECT * FROM STOCK WHERE product_id = " + str(id_line)
        cursor.execute(stock_query)
        stock_result = cursor.fetchone()
        stock = stock_result[1] if stock_result else 0
        max = stock_result[2]
        unit = stock_result[3]

        # Obtenha o nome da categoria a partir da tabela "PRODUCT_CATEGORIES"
        category_query = "SELECT CATEGORY_NAME FROM PRODUCT_CATEGORIES WHERE category_id = " + str(category_id)
        cursor.execute(category_query)
        category_result = cursor.fetchone()
        category_name = category_result[0] if category_result else ''

        cyph.write('CREATE (p'+ str(count) + ':Product {id:' + str(id_line) + ', Product_name:' + "'" + str(name) + "'" + ', SKU:' + "'" + str(sku) + "'" + ', Price:' + str(price) + ', Discount_id:' + str(discount_id) + ', Created_at:' + "'" + str(created_at) + "'" + ', Last_modified:' + "'" + str(modified_str) + "'" + ', Stock:' + str(stock) +  ', Max_stock_quantity:' + str(max) +  ', Unit:' + "'" + str(unit) + "'" + ', Category_id:' + str(category_id) + ', Category_name:' + "'" + str(category_name) + "'" +  '});'+'\n')
        count += 1

    cyph.write('\n\n')

- `Discount`

In [ ]:
# Criar nodos dos Discount
def createDiscountNodes(cursor, cyph):
    query = "SELECT * FROM DISCOUNT"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela DISCOUNT

    count = 1
    for l in result:
        id_line = l[0]
        name = l[1]
        description = l[2]
        percentage = l[3]
        status = l[4]
        created = l[5]
        modified_str = '2022-07-01 09:57:48'
        if l[6] is not None:
            modified_str = str(l[6])

        cyph.write('CREATE (dis'+ str (count) + ':Discount {id:' + str(id_line) + ', Discount_name:' + "'" +  str(name) + "'" + ', Discount_description:' + "'" +  str(description) + "'" + ', Discount_percent:' +  str(percentage) + ', is_active_status:' + "'" +  str(status) + "'" + ', Created_at:' + "'" +  str(created) + "'" + ', Modified_at:' + "'" +  str(modified_str) + "'" +'});'+'\n')
        count += 1

    cyph.write('\n\n')

- `Cart_Item`

In [ ]:
# Criar nodos dos Cart Item
def createCartItemNode(cursor, cyph):
    query = "SELECT * FROM CART_ITEM"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela CART_ITEM

    count = 1
    for l in result:
        id_item = l[0]
        session = l[1]
        product_id = l[2]
        quantity = l[3]
        created = l[4]
        
        modified_str = '2022-07-01 09:57:48'
        if l[5] is not None:
            modified_str = str(l[5])
       
        cyph.write('CREATE (c'+ str(count) + ':Cart_Item {id:' + str(id_item) + ', session:' + str(session) + ', product_id:' + str(product_id) + ', quantity:' + str(quantity) + ', created_at:' + "'" + str(created) + "'" + ', modified_at:' + "'" + modified_str + "'" + '});' + '\n')
        count += 1

    cyph.write('\n\n')

- `Shopping_Session`

In [ ]:
# Criar nodos dos Shopping Session
def createShoppingSessionNode(cursor, cyph):
    query = "SELECT * FROM SHOPPING_SESSION"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela SHOPPING_SESSION

    count = 1
    for l in result:
        session_id = l[0]
        user_id = l[1]
        created = l[2]
        modified_str = '2022-07-01 09:57:48'
        if l[3] is not None:
            modified_str = str(l[3])

        cyph.write('CREATE (s'+ str (count) + ':Shopping_Session {id:' + str(session_id) + ', user_id:' + str(user_id) + ', created_at:' + "'" + str(created) + "'" + ', modified_at:' + "'" + modified_str + "'" + '});' + '\n')
        count += 1

    cyph.write('\n\n')

- `Order_details`

In [ ]:
# Criar nodos dos Detalhes de Encomenda
def createOrderDetailsNode(cursor, cyph):
    query = "SELECT * FROM ORDER_DETAILS"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela ORDER_DETAILS

    count = 1
    for l in result:
        id_line = l[0]
        user_id = l[1]
        total = l[2]
        payment_id = l[3]
        shipping_method = l[4]
        delivery_addressID = l[5]
        created = l[6]
        
        modified_str = '2022-07-01 09:57:48'
        if l[7] is not None:
            modified_str = str(l[7])

        cyph.write('CREATE (o'+ str(count) + ':Order_Details {id:' + str(id_line) + ', user_id:' + str(user_id) + ', total:' + str(total) + ', payment_id:' + str(payment_id) + ', shipping_method:' + "'" + str(shipping_method) + "'" + ', delivery_address_ID:' + str(delivery_addressID) + ', created_at:' + "'" + str(created) + "'" + ', modified_at:' + "'" + modified_str + "'" + '});' + '\n')
        count += 1

    cyph.write('\n\n')


- `Order_items`

In [ ]:
# Create nodes for Order Items
def createOrderItemsNode(cursor, cyph):
    query = "SELECT * FROM ORDER_ITEMS"
    cursor.execute(query)
    result = cursor.fetchall()

    # Iterate through the rows of the ORDER_ITEMS table
    count = 1
    for l in result:
        order_items_id = l[0]
        order_details_id = l[1]
        product_id = l[2]
        created_at = l[3]
        modified_str = '2022-07-01 09:57:48'
        if l[4] is not None:
            modified_str = str(l[4])

        cyph.write(
            'CREATE (oi' + str(count) + ':Order_Items {order_items_id:' + str(order_items_id) + ', order_details_id:' + str(order_details_id) + ', product_id:' + str(product_id) + ', created_at:' + "'" + str(created_at) + "'" + ', modified_at:' + "'" + str(modified_str) + "'" + '});' + '\n'
        )
        count += 1

    cyph.write('\n\n')


- `Payment_details`

In [ ]:
# Criar nodos dos Detalhes de Pagamento
def createPaymentDetailsNode(cursor, cyph):
    query = "SELECT * FROM PAYMENT_DETAILS"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela PAYMENT_DETAILS

    count = 1
    for l in result:
        id_line = l[0]
        order_id = l[1]
        amount = l[2]
        provider = l[3]
        payment_status = l[4]
        created = l[5]
        
        modified_str = '2022-07-01 09:57:48'
        if l[6] is not None:
            modified_str = str(l[6])

        cyph.write('CREATE (pd'+ str(count) + ':Payment_Details {id:' + str(id_line) + ', order_id:' + str(order_id) + ', amount:' + str(amount) + ', provider:' + "'" + provider + "'" + ', payment_status:' + "'" + payment_status + "'" + ', created_at:' + "'" + str(created) + "'" + ', modified_at:' + "'" + modified_str + "'" + '});' + '\n')
        count += 1

    cyph.write('\n\n')

- `Employees`

In [ ]:
def createEmployeesNode(cursor, cyph):
    query = "SELECT * FROM EMPLOYEES"
    cursor.execute(query)
    result = cursor.fetchall()

    # Percorrer as linhas da tabela EMPLOYEES
    count = 1
    for l in result:
        id_line = l[0]
        first_name = l[1]
        last_name = l[3]
        date_of_birth = l[4]
        department = l[5]
        hire_date = l[6]
        salary = l[7]
        phone_number = l[8]
        email = l[9]
        ssn_number = l[10]
        middle_name = ''
        if l[2]:
            middle_name = str(l[2])
        manager_id = 0
        if l[11] is not None:
            manager_id = str(l[11])

        # Obter o histórico de emprego do funcionário a partir da tabela EMPLOYEES_ARCHIVE
        archive_query = "SELECT * FROM EMPLOYEES_ARCHIVE WHERE new_employee_id = " + str(id_line)
        cursor.execute(archive_query)
        archive_result = cursor.fetchall()

        # Criar um campo de histórico de emprego para cada registro no histórico
        history = []
        for archive in archive_result:
            if archive[3] is not None:
                # Se old_employee_ID
                event_date = archive[0]
                event_type = archive[1]
                user_name = archive[2]
                old_employee_id = archive[3]
                old_first_name = archive[4]
                old_middle_name = archive[5] if archive[5] else ''
                old_last_name = archive[6]
                old_date_of_birth = archive[7]
                old_department_id = archive[8]
                old_hire_date = archive[9]
                old_salary = archive[10]
                old_phone_number = archive[11]
                old_email = archive[12]
                old_ssn_number = archive[13]
                old_manager_id = 0
                print(archive[13])
                print(archive[14])
                if not math.isnan(archive[14]):
                    old_manager_id = archive[14]
                
                history_item = {
                    'event_date': event_date,
                    'event_type': event_type,
                    'user_name': user_name,
                    'old_employee_id': old_employee_id,
                    'old_first_name': old_first_name,
                    'old_middle_name': old_middle_name,
                    'old_last_name': old_last_name,
                    'old_date_of_birth': old_date_of_birth,
                    'old_department_id': old_department_id,
                    'old_hire_date': old_hire_date,
                    'old_salary': old_salary,
                    'old_phone_number': old_phone_number,
                    'old_email': old_email,
                    'old_ssn_number': old_ssn_number,
                    'old_manager_id': old_manager_id
                }
                
                # Adicionar o registro de histórico ao histórico completo do funcionário
                history.append(history_item)

        # Criar o nó do funcionário
        cyph.write(
            'CREATE (e' + str(count) + ':Employees {id:' + str(id_line) + ', first_name:' + "'" + str(
                first_name) + "'" + ', middle_name:' + "'" + str(middle_name) + "'" + ', last_name:' + "'" + str(
                last_name) + "'" + ', date_of_birth:' + "'" + str(date_of_birth) + "'" + ', department_id:' + str(
                department) + ', hire_date:' + "'" + str(hire_date) + "'" + ', salary:' + str(salary) + ', phone:' + "'" + str(
                phone_number) + "'" + ', email:' + "'" + str(email) + "'" + ', ssn_number:' + "'" + str(
                ssn_number) + "'" + ', manager_id:' + str(manager_id) + ', employment_history: ' + json.dumps(history) + '});' + '\n'
        )


        count += 1

    cyph.write('\n\n')


- Departments

In [ ]:
# Criar nodos dos Departamentos
def createDepartmentsNode(cursor, cyph):
    query = "SELECT * FROM DEPARTMENTS"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela DEPARTMENTS

    count = 1
    for l in result:
        id_line = l[0]
        department_name = l[1]
        manager_id = l[2]
        department_desc = l[3]

        # Escape single quotes in department_name and department_desc variables
        department_name = department_name.replace("'", "")
        department_desc = department_desc.replace("'", "")

        cyph.write('CREATE (d'+ str(count) + ':Departments {id:' + str(id_line) + ', department_name:' + "'" + str(department_name) + "'" + ', manager_id:' + str(manager_id) + ', department_desc:' + "'" + str(department_desc) + "'" + '});' + '\n')
        count += 1

    cyph.write('\n\n')

- `Addresses`

In [ ]:
# Criar nodos dos Endereços
def createAddressesNode(cursor, cyph):
    query = "SELECT * FROM ADDRESSES"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Percorrer as linhas da tabela ADDRESSES

    count = 1
    for l in result:
        id_line = l[0]
        line1 = l[1]
        line2 = ""
        if(l[2] is not None):
            line2 = l[2]
        city = l[3]
        zip_code = l[4]
        province = l[5]
        country = l[6]
  
        cyph.write('CREATE (a'+ str(count) + ':Addresses {id:' + str(id_line) + ', Line_1:' + "'" + str(line1) + "'" + ', Line_2:' + "'" + str(line2) + "'" + ', City:' + "'" + str(city) + "'" + ', Zip_code:' + "'" + str(zip_code) + "'" + ', Province:' + "'" + str(province) + "'" + ', Country:' + "'" + str(country) + "'" + '});' + '\n')
        count += 1

    cyph.write('\n\n')

# Relacionamentos

- Relacionamento: employee `WORKS_IN` department

In [ ]:
def create_employee_department_rel(cursor, cyph):
    query = "SELECT * FROM Employees"
    cursor.execute(query)
    result = cursor.fetchall()

    count = 1
    for row in result:
        employee_id = row[0]
        department_id = row[5]

        var_e = 'e' + str(count)
        var_d = 'd' + str(count)

        query = (
            'MATCH (' + var_e + ':Employees {id: ' + str(employee_id) + '}), '
            '(' + var_d + ':Departments {id: ' + str(department_id) + '}) '
            'CREATE (' + var_e + ')-[:WORKS_IN]->(' + var_d + ');\n'
        )
        cyph.write(query)
        count += 1

    cyph.write('\n\n') 

- Relacionamento: shopping_session `BELONGS_TO` user

In [ ]:
def create_session_user_rel(cursor, cyph):
    # Retrieve session and user data from the database
    query = "SELECT * FROM Shopping_Session"
    cursor.execute(query)
    result = cursor.fetchall()

    count = 1
    for row in result:
        session_id = row[0]
        user_id = row[1]

        var_s = 's' + str(count)
        var_u = 'u' + str(count)

        query = (
            'MATCH (' + var_s + ':Shopping_Session {id: ' + str(session_id) + '}), '
            '(' + var_u + ':Store_User {id: ' + str(user_id) + '}) '
            'CREATE (' + var_s + ')-[:BELONGS_TO]->(' + var_u + ');\n'
        )
        cyph.write(query)
        count += 1

    cyph.write('\n\n')

- Relacionamento: order `PLACED_BY` user

In [ ]:
# Criar a relação "PLACED_BY" entre os nodos Order_Details && Store_users
def create_order_placed_by_user_rel(cursor, cyph):
    queryO = "SELECT * FROM ORDER_DETAILS"
    cursor.execute(queryO)
    resultO = cursor.fetchall()

    count = 1
    for order in resultO:
        order_id = order[0]
        user_id = order[1]

        var_o = 'o' + str(count)
        var_u = 'u' + str(count)

        query = (
            'MATCH (' + var_o + ':Order_Details {id: ' + str(order_id) + '}), '
            '(' + var_u + ':Store_User {id: ' + str(user_id) + '}) '
            'CREATE (' + var_o + ')-[:PLACED_BY]->(' + var_u + ');\n'
        )
        cyph.write(query)
        count += 1

    cyph.write('\n\n')


Relacionamento: product `HAS_DISCOUNT`

In [ ]:
def create_product_discount_rel(cursor, cyph):
    query = "SELECT * FROM Product"
    cursor.execute(query)
    result = cursor.fetchall()

    count = 1
    for row in result:
        product_id = row[0]
        discount_id = 0
        if(row[5] is not None):
            discount_id = row[5]

        var_p = 'p' + str(count)
        var_d = 'd' + str(count)

        query = (
            'MATCH (' + var_p + ':Product {id: ' + str(product_id) + '}), '
            '(' + var_d + ':Discount {id: ' + str(discount_id) + '}) '
            'CREATE (' + var_p + ')-[:HAS_DISCOUNT]->(' + var_d + ');\n'
        )
        cyph.write(query)
        count += 1

    cyph.write('\n\n')

- Relacionamento: Payment_Details `RELATES_TO` order

In [ ]:
def create_payment_order_rel(cursor, cyph):
    # Retrieve payment and order data from the database
    query = "SELECT * FROM Payment_Details"
    cursor.execute(query)
    result = cursor.fetchall()

    count = 1
    for row in result:
        payment_id = row[0]
        order_id = row[1]

        var_pd = 'pd' + str(count)
        var_od = 'od' + str(count)

        query = (
            'MATCH (' + var_pd + ':Payment_Details {id: ' + str(payment_id) + '}), '
            '(' + var_od + ':Order_Details {id: ' + str(order_id) + '}) '
            'CREATE (' + var_pd + ')-[:RELATES_TO]->(' + var_od + ');\n'
        )
        cyph.write(query)
        count += 1

    cyph.write('\n\n')

- Relacionamento: order `SHIPPED_TO` address

In [ ]:
def create_order_address_rel(cursor, cyph):
    # Retrieve order and address data from the database
    query = "SELECT * FROM Order_Details"
    cursor.execute(query)
    result = cursor.fetchall()

    count = 1
    for row in result:
        order_id = row[0]
        address_id = row[5]

        var_od = 'od' + str(count)
        var_a = 'a' + str(count)

        query = (
            'MATCH (' + var_od + ':Order_Details {id: ' + str(order_id) + '}), '
            '(' + var_a + ':Addresses {id: ' + str(address_id) + '}) '
            'CREATE (' + var_od + ')-[:SHIPPED_TO]->(' + var_a + ');\n'
        )
        cyph.write(query)
        count += 1

    cyph.write('\n\n')

- Relacionamento: order `CONTAINS` order_items 

In [ ]:
def create_order_item_rel(cursor, cyph):
    # Retrieve order details and order items data from the database
    query = "SELECT * FROM Order_Items"
    cursor.execute(query)
    result = cursor.fetchall()

    count = 1
    for row in result:
        order_details_id = row[1]
        order_item_id = row[0]

        var_od = 'od' + str(count)
        var_oi = 'oi' + str(count)

        query = (
            'MATCH (' + var_od + ':Order_Details {id: ' + str(order_details_id) + '}), '
            '(' + var_oi + ':Order_Items {id: ' + str(order_item_id) + '}) '
            'CREATE (' + var_od + ')-[:CONTAINS]->(' + var_oi + ');\n'
        )
        cyph.write(query)
        count += 1

    cyph.write('\n\n')

- Relacionamento: cart_item `CONTAINS` product

In [ ]:
def create_cartItem_product_rel(cursor, cyph):
    query = "SELECT * FROM CART_ITEM"
    cursor.execute(query)
    result = cursor.fetchall()

    # Iterate over the rows of the CART_ITEM table
    count = 1
    for row in result:
        cart_item_id = row[0]
        product_id = row[2]

        # Create a relationship between the cart_item and product nodes
        cyph.write('MATCH (c:Cart_Item {id: ' + str(cart_item_id) + '}) '
                   'MATCH (p:Product {id: ' + str(product_id) + '}) '
                   'CREATE (c)-[:IS_A]->(p);\n')
        count += 1

    cyph.write('\n\n')

- Relacionamento: shopping_session `CONTAINS` cart_item

In [ ]:
def create_shoppingSession_cartItem_rel(cursor, cyph):
    query = "SELECT * FROM SHOPPING_SESSION"
    cursor.execute(query)
    result = cursor.fetchall()

    # Iterate over the rows of the SHOPPING_SESSION table
    count = 1
    for row in result:
        session_id = row[0]

        # Get the cart items associated with the shopping session
        cart_items_query = "SELECT * FROM CART_ITEM WHERE session_id = " + str(session_id)
        cursor.execute(cart_items_query)
        cart_items_result = cursor.fetchall()

        # Create a relationship between the shopping_session and cart_item nodes
        for cart_item_row in cart_items_result:
            cart_item_id = cart_item_row[0]
            cyph.write('MATCH (s:Shopping_Session {id: ' + str(session_id) + '}), (c:Cart_Item {id: ' + str(cart_item_id) + '}) CREATE (s)-[:CONTAINS]->(c);\n')

        count += 1

    cyph.write('\n\n')

In [ ]:
def deleteIDs(cyph):
    query = "MATCH (u:Store_User) REMOVE u.id;" + "\n"
    cyph.write(query)
    query = "MATCH (pc:Product_Category) REMOVE pc.id;" + "\n"
    cyph.write(query)
    query = "MATCH (p:Product) REMOVE p.id;" + "\n"
    cyph.write(query)
    query = "MATCH (dis:Discount) REMOVE dis.id;" + "\n"
    cyph.write(query)
    query = "MATCH (c:Cart_Item) REMOVE c.id;" + "\n"
    cyph.write(query)
    query = "MATCH (s:Shopping_Session) REMOVE s.id;" + "\n"
    cyph.write(query)
    query = "MATCH (o:Order_Details) REMOVE o.id;" + "\n"
    cyph.write(query)
    query = "MATCH (pd:Payment_Details) REMOVE pd.id;" + "\n"
    cyph.write(query)
    query = "MATCH (e:Employees) REMOVE e.id;" + "\n"
    cyph.write(query)
    query = "MATCH (d:Departments) REMOVE d.id;" + "\n"
    cyph.write(query)
    query = "MATCH (a:Addresses) REMOVE a.id;" + "\n"
    cyph.write(query)

### Escrever script de criação de nodos

In [ ]:
# Criação do nome do ficheiro a ser utilizado
fileName = "neoNodes" + ".txt"

# Abertura do Ficheiro 
cyph = open(fileName, "w")


try:
    # Criar nodo StoreUser
    createStoreUserNodes(cursor,cyph) 
    # Criar nodo Product
    createProductNodes(cursor, cyph)
    # Criar nodo Discount
    createDiscountNodes(cursor, cyph) 
    # Criar nodo CartItem
    createCartItemNode(cursor, cyph) 
    # Criar nodo ShoppingSession
    createShoppingSessionNode(cursor, cyph)
    # Criar nodo OrderDetails  
    createOrderDetailsNode(cursor, cyph) 
    # Criar nodo OrderItems
    createOrderItemsNode(cursor, cyph)
    # Criar nodo PaymentDetails
    createPaymentDetailsNode(cursor, cyph)
    # Criar nodo Employees
    createEmployeesNode(cursor, cyph) 
    # Criar nodo Departments
    createDepartmentsNode(cursor, cyph) 
    # Criar nodo Addresses
    createAddressesNode(cursor, cyph) 
    
    
    cyph.close()
    
except cx_Oracle.DatabaseError as er:
    print('ERROR - Oracle Database:', er)

except:
    print("ERROR - Node Creation.")

### Escrever script de Relacionamentos

In [ ]:
# Open the file in write mode
cyph_file = open("neoRelationships.txt", "w")

try:
    # Criar Relações:
    create_employee_department_rel(cursor, cyph_file)
    create_session_user_rel(cursor, cyph_file)
    create_order_placed_by_user_rel(cursor, cyph_file)
    create_product_discount_rel(cursor, cyph_file)
    create_payment_order_rel(cursor, cyph_file)
    create_order_address_rel(cursor, cyph_file)
    create_cartItem_product_rel(cursor, cyph_file)
    create_shoppingSession_cartItem_rel(cursor, cyph_file)
    create_order_item_rel(cursor, cyph_file)
    
    deleteIDs(cyph_file)
    cyph_file.close()
except Exception as e:
    print('ERROR - Relationship Creation')
    print(e)


### Estabelecer conexão com o Neo4J

In [ ]:
from neo4j import GraphDatabase

# Establish a connection to the Neo4j database
uri = 'bolt://localhost:7687'
driver = GraphDatabase.driver(uri, auth=('neo4j', 'uminho2023'))

### Utility functions

In [30]:
# Function to execute Cypher queries
def execute_cypher_query(query):
    with driver.session() as session:
        session.run(query)


def delete_all_data():
    query = "MATCH (n) DETACH DELETE n"

    with driver.session() as session:
        session.run(query)

### Criação de nodos e relacionamentos

In [32]:
# Create Nodes
with open("neoNodes.txt", "r") as file:
    queries = file.readlines()
    for query in queries:
        trimmed_query = query.strip()
        if trimmed_query:
            execute_cypher_query(trimmed_query)

# Create Relationships
with open("neoRelationships.txt", "r") as file:
    queries = file.readlines()
    for query in queries:
        trimmed_query = query.strip()
        if trimmed_query:
            execute_cypher_query(trimmed_query)

# Close the database connection
driver.close()